In [1]:
import uuid
import pandas
import numpy
from random import shuffle

import sklearn
import sklearn.ensemble

from sklearn.ensemble import RandomForestRegressor

import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import os

In [2]:
from sklearn import preprocessing
import numpy

In [3]:
def load_data():
    
    filename = "/Users/simondi/PHD/data/data/target_infect_x/screening_data_subset/cells_sample_10_normalized_cut.tsv"

    data = pandas.read_csv(filename, sep="\t", header=0)

    feature_cols = [(i, x) for i, x in enumerate(data.columns.values) if
                    x.startswith("cells")]
    for i, c in feature_cols:
        data.loc[:, c] = data.loc[:, c].astype('float64')

    X = data.dropna(subset=["cells.children_invasomes_count", "cells.children_bacteria_count"], how='all')
    X["cells.children_invasomes_count"] =  X["cells.children_invasomes_count"].fillna(0)
    X["cells.children_bacteria_count"] =  X["cells.children_bacteria_count"].fillna(0)
    X = X.dropna()
    X["infection"] = X["cells.children_invasomes_count"] + X["cells.children_bacteria_count"] 
    del X["cells.children_invasomes_count"]
    del X["cells.children_bacteria_count"]
    return X

In [4]:
data = load_data()

/Users/simondi/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/simondi/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
df = data.groupby(["pathogen", "gene"]).apply(lambda x: x.sample(frac=0.01))

In [12]:
df

study    pathogen library design  \
pathogen   gene                                                             
bartonella a1cf     4126711            infectx  bartonella       d      p   
           a2ld1    1016343            infectx  bartonella       q      u   
           a2m      4337476            infectx  bartonella       q      u   
           a4gnt    5002229            infectx  bartonella       q      u   
           aaas     3370605            infectx  bartonella       d      p   
           aadac    1725982            infectx  bartonella       q      u   
           aadat    1054701            infectx  bartonella       q      u   
           aagab    966474             infectx  bartonella       q      u   
           aak1     5625412  infectx_published  bartonella       q      u   
                    2556391            infectx  bartonella       q      u   
           aamp     2636166            infectx  bartonella       q      u   
           aanat    2636644            infectx  bartonella       q      u   
           aars     3790166            infectx  bartonella       q      u   
           aars2    1554762            infectx  bartonella       d      p   
           aasdh    5185404            infectx  bartonella       q      u   
           aasdhppt 5194573            infectx  bartonella       d      p   
           aass     3363626            infectx  bartonella       d      p   
           aatk     3879185  infectx_published  bartonella       q      u   
           abat     2889775            infectx  bartonella       d      p   
           abca1    1614622            infectx  bartonella       q      u   
           abca12   4677683            infectx  bartonella       q      u   
           abca13   3893435            infectx  bartonella       q      u   
           abca2    3053357            infectx  bartonella       d      p   
           abca3    4836321            infectx  bartonella       q      u   
           abca5    2854565            infectx  bartonella       q      u   
           abca6    1455459            infectx  bartonella       q      u   
           abca8    3642090            infectx  bartonella       q      u   
           abca9    2248958            infectx  bartonella       q      u   
           abcb1    3268535            infectx  bartonella       d      p   
           abcb10   5496208            infectx  bartonella       q      u   
...                                        ...         ...     ...    ...   
salmonella vrk2     3739242  infectx_published  salmonella       d      u   
           vrk3     4925730  infectx_published  salmonella       a      u   
                    3907260            infectx  salmonella       d      p   
           wdsub1   3451917            infectx  salmonella       d      p   
           wee1     3011524  infectx_published  salmonella       a      u   
                    3011530  infectx_published  salmonella       a      u   
           wnk1     1491789  infectx_published  salmonella       d      u   
                    4492368            infectx  salmonella       d      p   
           wnk2     5484979            infectx  salmonella       d      p   
                    4883796  infectx_published  salmonella       q      u   
           wnk3     4501144  infectx_published  salmonella       q      u   
                    218585   infectx_published  salmonella       q      u   
           wnk4     4928252  infectx_published  salmonella       a      u   
                    4599187  infectx_published  salmonella       q      u   
           wwp1     5670155            infectx  salmonella       d      p   
           xrcc6bp1 3909858            infectx  salmonella       d      p   
           xylb     2050473  infectx_published  salmonella       d      u   
                    3896664  infectx_published  salmonella       d      u   
           yes1     1276600  infectx_published  salmonella       q      u   
                    4159592          

In [6]:
n=10
rf = RandomForestRegressor(n_estimators=n, max_features='sqrt')

In [7]:
feature_cols = [x for x in df.columns.values if x.startswith("cells")]
trees=rf.fit(X=df.loc[:,feature_cols], y=df.loc[:,"infection"])

In [10]:
zippedf = [(x,y) for x, y in zip(feature_cols, trees.feature_importances_)]

In [26]:
zippedf = sorted(zippedf, key=lambda x: -x[1]) 

In [15]:
uid = str(uuid.uuid1())
flout= "/Users/simondi/PROJECTS/target_infect_x_project/src/tix_util/tix_analysis/results/importance"

In [24]:
flout = flout + "_trees_" + str(n)+ "_" + uid + ".txt"

In [27]:
with open(flout, 'w') as fh:
    for z in zippedf:
        fh.write(z[0] +"\t"  + "{0:.2f}".format(z[1]) + "\n")